In [1]:
! pip install ucimlrepo


   -------------------- ------------------- 1/2 [ucimlrepo]
   ---------------------------------------- 2/2 [ucimlrepo]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
computer_hardware = fetch_ucirepo(id=29) 
  
# data (as pandas dataframes) 
X = computer_hardware.data.features 
y = computer_hardware.data.targets 
  
# metadata 
print(computer_hardware.metadata) 
  
# variable information 
print(computer_hardware.variables) 


{'uci_id': 29, 'name': 'Computer Hardware', 'repository_url': 'https://archive.ics.uci.edu/dataset/29/computer+hardware', 'data_url': 'https://archive.ics.uci.edu/static/public/29/data.csv', 'abstract': 'Relative CPU Performance Data, described in terms of its cycle time, memory size, etc.', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 209, 'num_features': 10, 'feature_types': ['Integer'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1987, 'last_updated': 'Fri Oct 27 2023', 'dataset_doi': '10.24432/C5830D', 'creators': ['Jacob Feldmesser'], 'intro_paper': None, 'additional_info': {'summary': 'The estimated relative performance values were estimated by the authors using a linear regression method.  See their article (pp 308-313) for more details on how the relative performance values were set.', 'purpose': None, 'funded_by': N

In [5]:
import pandas as p
X.head()

X.describe()
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   VendorName  209 non-null    object
 1   ModelName   209 non-null    object
 2   MYCT        209 non-null    int64 
 3   MMIN        209 non-null    int64 
 4   MMAX        209 non-null    int64 
 5   CACH        209 non-null    int64 
 6   CHMIN       209 non-null    int64 
 7   CHMAX       209 non-null    int64 
 8   PRP         209 non-null    int64 
 9   ERP         209 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 16.5+ KB


In [7]:
X.head()

,VendorName,ModelName,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


In [10]:
X.drop(columns=['ModelName'], inplace = True)

In [11]:
X['VendorName'].value_counts()

VendorName
ibm             32
nas             19
ncr             13
honeywell       13
sperry          13
siemens         12
amdahl           9
cdc              9
burroughs        8
hp               7
harris           7
dg               7
c.r.d            6
dec              6
ipl              6
magnuson         6
prime            5
cambex           5
formation        5
gould            3
nixdorf          3
perkin-elmer     3
basf             2
wang             2
bti              2
apollo           2
four-phase       1
adviser          1
microdata        1
sratus           1
Name: count, dtype: int64

In [8]:
X['ModelName'].value_counts()

ModelName
32/60            1
470v/7           1
470v/7a          1
470v/7b          1
470v/7c          1
                ..
80/8             1
90/80-model-3    1
32               1
vs-100           1
vs-90            1
Name: count, Length: 209, dtype: int64

In [13]:
df = X.copy()
vendor_freq = df['VendorName'].value_counts()
df['Vendor_freq'] = df['VendorName'].map(vendor_freq)


In [14]:
df.drop(columns=['VendorName'], inplace = True)

In [15]:
df.head()

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP,Vendor_freq
0,125,256,6000,256,16,128,198,199,1
1,29,8000,32000,32,8,32,269,253,9
2,29,8000,32000,32,8,32,220,253,9
3,29,8000,32000,32,8,32,172,253,9
4,29,8000,16000,32,8,16,132,132,9


In [19]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# جلب البيانات
data = fetch_ucirepo(id=29)
df = pd.concat([data.data.features, data.data.targets], axis=1)

# فصل الميزات والهدف (EPR)
X = df.drop(columns=['ERP', 'VendorName', 'ModelName'], errors='ignore')
y = df['ERP']

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# بناء النموذج
model = LinearRegression()
model.fit(X_train, y_train)

# التوقع
y_pred = model.predict(X_test)

# الأداء
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("🔹 Mean Squared Error (MSE):", round(mse, 2))
print("🔹 R² Score:", round(r2, 2))

# جدول مقارنة النتائج
comparison_df = pd.DataFrame({'Actual ERP': y_test.values, 'Predicted ERP': np.round(y_pred, 2)})
print("\n🔸 مقارنة بين القيمة الحقيقية والمتوقعة:")
print(comparison_df.reset_index(drop=True).head(20))  # عرض أول 20 فقط


🔹 Mean Squared Error (MSE): 3007.89
🔹 R² Score: 0.94

🔸 مقارنة بين القيمة الحقيقية والمتوقعة:
    Actual ERP  Predicted ERP
0          102         162.48
1           25          11.59
2           25          11.92
3          919         768.16
4           34          37.85
5          267         311.32
6           41          50.15
7           19         -10.29
8         1238         943.87
9          227         230.59
10          88          85.48
11          99         126.00
12          41          41.72
13          44          36.46
14          64          66.99
15          23          16.16
16          80          56.78
17          22           3.87
18         142         135.01
19          82         116.65


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# pipeline with scaling
model = make_pipeline(StandardScaler(), LinearRegression())
model.fit(X_train, y_train)
y_pred_scaled = model.predict(X_test)

print("With Scaling:")
print("R² Score:", r2_score(y_test, y_pred_scaled))
print("MSE:", mean_squared_error(y_test, y_pred_scaled))


With Scaling:
R² Score: 0.9440465034138787
MSE: 3007.8898321639335


In [17]:
df.describe()

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP,Vendor_freq
count,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000
mean,203.822967,2867.980861,11796.153110,25.205742,4.698565,18.267943,105.622010,99.330144,12.799043
std,260.262926,3878.742758,11726.564377,40.628722,6.816274,25.997318,160.830733,154.757102,9.319015
min,17.000000,64.000000,64.000000,0.000000,0.000000,0.000000,6.000000,15.000000,1.000000
25%,50.000000,768.000000,4000.000000,0.000000,1.000000,5.000000,27.000000,28.000000,6.000000
50%,110.000000,2000.000000,8000.000000,8.000000,2.000000,8.000000,50.000000,45.000000,9.000000
75%,225.000000,4000.000000,16000.000000,32.000000,6.000000,24.000000,113.000000,101.000000,13.000000
max,1500.000000,32000.000000,64000.000000,256.000000,52.000000,176.000000,1150.000000,1238.000000,32.000000


In [18]:
# ✅ Import Libraries
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# ✅ Load Dataset
data = fetch_ucirepo(id=29)
df = pd.concat([data.data.features, data.data.targets], axis=1)

# ✅ Simple EDA
print(df.head())
print(df.info())
print(df.describe())

# ✅ Feature Engineering: encode 'VendorName'
le = LabelEncoder()
df['Vendor_encoded'] = le.fit_transform(df['VendorName'])

# ✅ Drop 'ModelName' (too specific, high cardinality)
df = df.drop(columns=['VendorName', 'ModelName'])

# ✅ Split Features and Target
X = df.drop(columns=['PRP'])
y = df['PRP']

# ✅ Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Train Model
model = LinearRegression()
model.fit(X_train, y_train)

# ✅ Predict
y_pred = model.predict(X_test)

# ✅ Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🔹 Mean Squared Error (MSE): {mse:.2f}")
print(f"🔹 R² Score: {r2:.2f}")

# ✅ Optional: Show some predictions
results = pd.DataFrame({
    'Actual PRP': y_test.values,
    'Predicted PRP': y_pred
})
print(results.head())


  VendorName ModelName  MYCT  MMIN   MMAX  CACH  CHMIN  CHMAX  PRP  ERP
0    adviser     32/60   125   256   6000   256     16    128  198  199
1     amdahl    470v/7    29  8000  32000    32      8     32  269  253
2     amdahl   470v/7a    29  8000  32000    32      8     32  220  253
3     amdahl   470v/7b    29  8000  32000    32      8     32  172  253
4     amdahl   470v/7c    29  8000  16000    32      8     16  132  132
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   VendorName  209 non-null    object
 1   ModelName   209 non-null    object
 2   MYCT        209 non-null    int64 
 3   MMIN        209 non-null    int64 
 4   MMAX        209 non-null    int64 
 5   CACH        209 non-null    int64 
 6   CHMIN       209 non-null    int64 
 7   CHMAX       209 non-null    int64 
 8   PRP         209 non-null    int64 
 9   ERP         209 no